In [20]:
import findspark
findspark.init()

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.streaming import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import os

spark = SparkSession.builder.master("local").appName("multi").getOrCreate()
sc = spark.sparkContext

In [22]:
# sql adaptive query execution adaptive.coalescePartitions.enabled will makr paritions dynamic
sc.setLogLevel("Error")
spark.conf.set("spark.sql.shuffle.partitions",3)
spark.conf.get("spark.sql.shuffle.partitions")
spark.conf.set("spark.sql.adaptive.enabled","false")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled","false")
spark.conf.set("spark.sql.execution.arrow.enabled",True)
spark.conf.set("spark.sql.execution.arrow.fallback.enabled",True)

"""
Multi folder read
"""

#WithoutInferSchema
#headerTrue will read first row and assign column names but type is String for all
#spark job created to read first column
filepath = "file:///C:/Users/venka/PycharmProjects/pythonProject/dataset/"
df = spark.read.option("header",True).csv(filepath + "Multi/Data1/file1.csv")
df.show()

+----+---+---------+----+
|Name|age|education|Year|
+----+---+---------+----+
|Mike| 28|       BE|2014|
|Jack| 43|      MBA|2000|
+----+---+---------+----+



In [8]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- education: string (nullable = true)
 |-- Year: string (nullable = true)



In [25]:
#Method1 List inputpath
filepaths = ["Multi/Data1/*.csv","Multi/Data2/*.csv","Multi/Data3/*.csv","Multi/DataSchema/*.csv"]
full_filepaths = [os.path.join(filepath, path) for path in filepaths]
df1 = spark.read.option("header",True) \
        .csv(full_filepaths)
df1.show()

+----------------+----+---------+----+
|            Name| age|education|Year|
+----------------+----+---------+----+
|           Gayle|  40|      BBA|2003|
|           Rahul|  24|     B.Sc|2012|
|           Dhoni|  38|      MBA|2005|
|           Rinku|  26|     B.Ed|2016|
|           Rohit|  32|       MS|2010|
|          Pollar|  36|     M.Sc|2010|
|           Kolhi|  29|       ME|2017|
|          Hussey|  43|     Ph.d|2013|
|          Parker|  23|       BE|2019|
|           Peter|  29|       ME|2013|
|            Mike|  28|       BE|2014|
|            Jack|  43|      MBA|2000|
|         age:Int|NULL|     NULL|NULL|
|education:String|NULL|     NULL|NULL|
|        Year:Int|NULL|     NULL|NULL|
+----------------+----+---------+----+



In [28]:
#Method2 Use regex pattern
df2=spark.read.option("header",True).csv(filepath+"Multi/Data[1-3]*/*.csv")
df2.show()

+------+---+---------+----+
|  Name|age|education|Year|
+------+---+---------+----+
| Gayle| 40|      BBA|2003|
| Rahul| 24|     B.Sc|2012|
| Dhoni| 38|      MBA|2005|
| Rinku| 26|     B.Ed|2016|
| Rohit| 32|       MS|2010|
|Pollar| 36|     M.Sc|2010|
| Kolhi| 29|       ME|2017|
|Hussey| 43|     Ph.d|2013|
|Parker| 23|       BE|2019|
| Peter| 29|       ME|2013|
|  Mike| 28|       BE|2014|
|  Jack| 43|      MBA|2000|
+------+---+---------+----+



In [29]:
#Method3 with curly braces
df3=spark.read.option("header",True).csv(filepath+"Multi/Data{1,2,3}*/*.csv")
df3.show()

+------+---+---------+----+
|  Name|age|education|Year|
+------+---+---------+----+
| Gayle| 40|      BBA|2003|
| Rahul| 24|     B.Sc|2012|
| Dhoni| 38|      MBA|2005|
| Rinku| 26|     B.Ed|2016|
| Rohit| 32|       MS|2010|
|Pollar| 36|     M.Sc|2010|
| Kolhi| 29|       ME|2017|
|Hussey| 43|     Ph.d|2013|
|Parker| 23|       BE|2019|
| Peter| 29|       ME|2013|
|  Mike| 28|       BE|2014|
|  Jack| 43|      MBA|2000|
+------+---+---------+----+

